In [2]:
# getiing data from json file 
import json
import codecs

f = open('goodreads_reviews_children.json','r')

fw1 = codecs.open('goodReads_reviews.txt','w',encoding='utf8')
fw2 = codecs.open('badReads_reviews.txt','w',encoding='utf8')

for i in f:
    df = json.loads(i)
    reviewData,rating = 'N/A','N/A'
    if int(df['rating']) == 4 or int(df['rating']) == 5:
        reviewData = df['review_text']
        reviewData = reviewData.replace('\n','')
        reviewData = reviewData.replace('\t',' ')
        reviewData = reviewData.strip()
        rating = '1' #good
        fw1.write(reviewData+'\t'+rating+'\n')
    elif int(df['rating']) == 0 or int(df['rating']) == 1 or int(df['rating']) == 2:
        reviewData = df['review_text']
        reviewData = reviewData.replace('\n','')
        reviewData = reviewData.replace('\t',' ')
        reviewData = reviewData.strip()
        rating = '0' #bad
        fw2.write(reviewData+'\t' +rating+'\n')

fw1.close()
fw2.close()


In [8]:
# To Split data into training and testing if the data is in the form "list" of strings by using sklearn library
from sklearn.model_selection import train_test_split
fw3 = codecs.open('train.txt','w',encoding='utf8')
fw4 = codecs.open('test.txt','w',encoding='utf8')

def loadData(fName): #Creating Training Data and Testing Data
    reviews = []
    labels = []
    f2=open(fName)
    for line in f2:
        review = line.strip() 
        reviews.append(review) #Storing Data in a list
    
    f2.close()

    train, test = train_test_split(reviews, test_size = 0.25)
    
    fw3.write("\n".join(train))
    fw4.write('\n'.join(test))
    
loadData('goodReads_reviews.txt')
loadData('badReads_reviews.txt')

fw3.close()
fw4.close()


In [9]:
#Based on Train and Test file we have to create training and testing variables to train the model based on the reviews with the corresponding labels.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


def getTrainTestData(fName):
    reviews=[]
    labels=[]
    f3=open(fName)
    for line in f3:
        review,rating=line.strip().split('\t')  
        reviews.append(review.lower())    
        labels.append(int(rating))
    f3.close()
    return reviews,labels


rev_train,labels_train=getTrainTestData('train.txt')
rev_test,labels_test=getTrainTestData('test.txt')

#Build a counter based on the training dataset
counter = CountVectorizer(lowercase=True, stop_words = None)
counter.fit(rev_train)


#count the number of times each term appears in a document and transform each doc into a count vector
counts_train = counter.transform(rev_train)#transform the training data
counts_test = counter.transform(rev_test)#transform the testing data

#train classifier
clf= LogisticRegression(solver='liblinear', dual=False, fit_intercept=True, intercept_scaling=1,tol=0.0001, C=1.0)

#train all classifier on the same datasets
clf.fit(counts_train,labels_train)

#use hard voting to predict (majority voting)
pred=clf.predict(counts_test)

#print accuracy
print (accuracy_score(pred,labels_test))

ValueError: not enough values to unpack (expected 2, got 1)